In [6]:
import os
import ast
import pandas as pd
import shutil

In [2]:
filename = "FL_Restaurants_Photos" + ".csv"
path = "C:/Users/tokud/OneDrive/Documents/Machine Learning - UCSD Extension Springboard Course/Capstone/Datasets/" + filename

merged_photos_df = pd.read_csv(path)
merged_photos_df.shape

(30410, 17)

In [3]:
label_count = merged_photos_df['label'].value_counts()
print(label_count)

food       16895
inside      7428
outside     3608
drink       2201
menu         278
Name: label, dtype: int64


In [69]:
food_df = merged_photos_df[merged_photos_df['label'] == 'food']
food_df.shape

(16895, 17)

In [5]:
food_df[['name', 'attributes', 'categories']].head()

,name,attributes,categories
1,Zio's Italian Market,"{'OutdoorSeating': 'False', 'RestaurantsGoodForGroups': 'False', 'WiFi': ""u'no'"", 'RestaurantsPriceRange2': '1', 'RestaurantsDelivery': 'True', 'RestaurantsAttire': ""u'casual'"", 'BusinessAcceptsCreditCards': 'True', 'RestaurantsTakeOut': 'True', 'Caters': 'True', 'NoiseLevel': ""u'average'"", 'Ambience': ""{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}"", 'GoodForKids': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'BikeParking': 'True', 'RestaurantsTableService': 'False', 'Alcohol': ""u'none'"", 'RestaurantsReservations': 'False', 'GoodForMeal': ""{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}"", 'HasTV': 'True', 'WheelchairAccessible': 'True'}","Food, Delis, Italian, Bakeries, Restaurants"
2,Zio's Italian Market,"{'OutdoorSeating': 'False', 'RestaurantsGoodForGroups': 'False', 'WiFi': ""u'no'"", 'RestaurantsPriceRange2': '1', 'RestaurantsDelivery': 'True', 'RestaurantsAttire': ""u'casual'"", 'BusinessAcceptsCreditCards': 'True', 'RestaurantsTakeOut': 'True', 'Caters': 'True', 'NoiseLevel': ""u'average'"", 'Ambience': ""{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}"", 'GoodForKids': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'BikeParking': 'True', 'RestaurantsTableService': 'False', 'Alcohol': ""u'none'"", 'RestaurantsReservations': 'False', 'GoodForMeal': ""{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}"", 'HasTV': 'True', 'WheelchairAccessible': 'True'}","Food, Delis, Italian, Bakeries, Restaurants"
5,Roman Forum,"{'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'BikeParking': 'False', 'OutdoorSeating': 'True', 'RestaurantsDelivery': 'False', 'GoodForKids': 'True', 'RestaurantsTakeOut': 'True', 'BusinessAcceptsCreditCards': 'True', 'Caters': 'True', 'NoiseLevel': ""u'quiet'"", 'WiFi': ""'no'"", 'HasTV': 'True', 'Ambience': ""{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}"", 'RestaurantsAttire': ""u'casual'"", 'RestaurantsReservations': 'True', 'Alcohol': ""u'full_bar'"", 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2'}","Restaurants, American (New), Italian"
6,Top Shelf Sports Lounge,"{'BestNights': ""{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}"", 'ByAppointmentOnly': 'False', 'GoodForDancing': 'False', 'BusinessAcceptsCreditCards': 'True', 'HappyHour': 'True', 'Alcohol': ""u'full_bar'"", 'RestaurantsAttire': ""u'casual'"", 'Smoking': ""u'outdoor'"", 'BikeParking': 'True', 'CoatCheck': 'False', 'HasTV': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsGoodForGroups': 'True', 'Ambience': ""{'touristy': False, 'hipster': None, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': None, 'upscale': None, 'classy': False, 'casual': True}"", 'Music': ""{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': None, 'live': False, 'video': False, 'karaoke': None}"", 'RestaurantsTableService': 'True', 'BYOB': 'False', 'WiFi': ""u'free'"", 'DogsAllowed': 'False', 'Caters': 'True', 'WheelchairAccessible': 'True', 'NoiseLevel': ""u'average'"", 'DriveThru': 'False', 'OutdoorSeating': 'False', 'RestaurantsReservations': 'True', 'BusinessAcceptsBitcoin': 'False', 'RestaurantsTakeOut': 'True', 'GoodForKids': 'False', 'BusinessParking': 'N

In [20]:
def upscale_attribute(attributes_str):
    try:
        attributes = ast.literal_eval(attributes_str)

        if 'Ambience' in attributes and attributes['Ambience']:
            ambience = ast.literal_eval(attributes['Ambience'])
            
            if isinstance(ambience, dict):
                upscale = ambience.get('upscale') is True
                classy = ambience.get('classy') is True
                return upscale and classy
    except (ValueError, SyntaxError, TypeError):
        pass
    return False

In [21]:
food_df['is_upscale'] = food_df['attributes'].apply(upscale_attribute)

upscale_df = food_df[food_df['is_upscale'] == True]
upscale_df.shape

C:\Users\tokud\AppData\Local\Temp/ipykernel_22740/3568253813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_df['is_upscale'] = food_df['attributes'].apply(upscale_attribute)


(536, 18)

In [26]:
photos_dir = "C:/Users/tokud/OneDrive/Documents/Machine Learning - UCSD Extension Springboard Course/Capstone/Datasets/Food/"
output_dir = "C:/Users/tokud/OneDrive/Documents/Machine Learning - UCSD Extension Springboard Course/Capstone/Datasets/Upscale Food/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

photo_ids = set(upscale_df['photo_id'].tolist())  


for image_file in os.listdir(photos_dir):
    if image_file.endswith('.jpg'):

        photo_id = os.path.splitext(image_file)[0]
        
        if photo_id in photo_ids:
            src_path = os.path.join(photos_dir, image_file)
            dest_path = os.path.join(output_dir, image_file)
            
            shutil.copy(src_path, dest_path)
            

In [98]:
categories = food_df['categories'].value_counts()
print(categories[0:25])

Poke, Restaurants, Sushi Bars, Food                                                                                                                                                                         98
Restaurants, Breakfast & Brunch, Delis, Cafes, Event Planning & Services, Caterers                                                                                                                          82
Italian, Restaurants                                                                                                                                                                                        79
Mexican, Restaurants                                                                                                                                                                                        79
Restaurants, Sushi Bars                                                                                                                                                     

In [32]:
categories = food_df['categories'].value_counts()
category_list = categories.index.tolist()
print(category_list)

['Poke, Restaurants, Sushi Bars, Food', 'Restaurants, Breakfast & Brunch, Delis, Cafes, Event Planning & Services, Caterers', 'Italian, Restaurants', 'Mexican, Restaurants', 'Restaurants, Sushi Bars', 'Breweries, Food, American (New), Beer, Wine & Spirits, Restaurants, Seafood', 'Restaurants, Pizza', 'Restaurants, Burgers, Fast Food', 'Sushi Bars, Nightlife, Seafood, Thai, Restaurants, Bars', 'Burgers, Bars, Restaurants, American (Traditional), Vegan, Asian Fusion, Cocktail Bars, Breakfast & Brunch, Soul Food, Sandwiches, American (New), Gastropubs, Comfort Food, Nightlife, Wine Bars, Delis', 'Restaurants, Chinese', 'Restaurants, Thai', 'Restaurants, American (New)', 'Tapas/Small Plates, Gastropubs, Restaurants, Spanish, Tapas Bars, Mediterranean', 'Asian Fusion, Chinese, Restaurants, Szechuan', 'Cafes, American (Traditional), Breakfast & Brunch, Restaurants', 'Restaurants, Mexican', 'Russian, Restaurants, Breakfast & Brunch, Comfort Food, Ukrainian, Polish, Modern European', 'Seafood,

In [38]:
keywords = ['fast food', 'burgers', 'hot dogs', 'pizza', 'hamburgers', 'drive-thru', 'fries']

filtered_categories = [category for category in category_list if any(keyword.lower() in category.lower() for keyword in keywords)]

In [70]:
fastfood_df = food_df[food_df['categories'].apply(lambda category: any(keyword.lower() in category.lower() for keyword in keywords))]
fastfood_df.shape

(4341, 17)

In [73]:
def fastfood_attribute(attributes_str):
    if pd.isna(attributes_str):
        return False
    try:
        attributes = ast.literal_eval(attributes_str)
        
        if attributes.get('RestaurantsTableService') == 'False' and attributes.get('DriveThru') == 'True':
            return True
    except (ValueError, SyntaxError, TypeError):
        pass
    return False

In [74]:
fastfood_copy = fastfood_df.copy()

In [75]:
fastfood_copy['is_fastfood'] = fastfood_copy['attributes'].apply(fastfood_attribute)

filtered_df = fastfood_copy[fastfood_copy['is_fastfood'] == True]
filtered_df.shape

(329, 18)

In [77]:
photos_dir = "C:/Users/tokud/OneDrive/Documents/Machine Learning - UCSD Extension Springboard Course/Capstone/Datasets/Food/"
output_dir = "C:/Users/tokud/OneDrive/Documents/Machine Learning - UCSD Extension Springboard Course/Capstone/Datasets/Fast Food/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

photo_ids = set(filtered_df['photo_id'].tolist())  


for image_file in os.listdir(photos_dir):
    if image_file.endswith('.jpg'):

        photo_id = os.path.splitext(image_file)[0]
        
        if photo_id in photo_ids:
            src_path = os.path.join(photos_dir, image_file)
            dest_path = os.path.join(output_dir, image_file)
            
            shutil.copy(src_path, dest_path)

In [80]:
pizza_df = fastfood_copy[fastfood_copy['categories'].str.contains('pizza', case=False, na=False)]
pizza_df.shape

(1388, 18)

In [82]:
def pizza_attribute(attributes_str):
    if pd.isna(attributes_str):
        return False
    try:
        attributes = ast.literal_eval(attributes_str)
        
        if attributes.get('RestaurantsTableService') == 'False' and attributes.get('RestaurantsDelivery') == 'True':
            return True
    except (ValueError, SyntaxError, TypeError):
        pass
    return False

In [83]:
pizza_df_copy = pizza_df.copy()

In [84]:
pizza_df_copy['is_delivery'] = pizza_df_copy['attributes'].apply(pizza_attribute)

pizza_filtered = pizza_df_copy[pizza_df_copy['is_delivery'] == True]
pizza_filtered.shape

(192, 19)

In [85]:
photos_dir = "C:/Users/tokud/OneDrive/Documents/Machine Learning - UCSD Extension Springboard Course/Capstone/Datasets/Food/"
output_dir = "C:/Users/tokud/OneDrive/Documents/Machine Learning - UCSD Extension Springboard Course/Capstone/Datasets/Pizza/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

photo_ids = set(pizza_filtered['photo_id'].tolist())  


for image_file in os.listdir(photos_dir):
    if image_file.endswith('.jpg'):

        photo_id = os.path.splitext(image_file)[0]
        
        if photo_id in photo_ids:
            src_path = os.path.join(photos_dir, image_file)
            dest_path = os.path.join(output_dir, image_file)
            
            shutil.copy(src_path, dest_path)

In [94]:
burritos_df = fastfood_copy[fastfood_copy['categories'].str.contains('Mexican|Chinese', case=False, na=False)]
burritos_df.shape

(397, 18)

In [95]:
def otherfood_attribute(attributes_str):
    if pd.isna(attributes_str):
        return False
    try:
        attributes = ast.literal_eval(attributes_str)
        
        if attributes.get('RestaurantsTableService') == 'False':
            return True
    except (ValueError, SyntaxError, TypeError):
        pass
    return False

In [96]:
burritos_copy = burritos_df.copy()

burritos_copy['is_fastfood'] = burritos_copy['attributes'].apply(otherfood_attribute)

burritos_filtered = burritos_copy[burritos_copy['is_fastfood'] == True]
burritos_filtered.shape

(80, 18)

In [97]:
photos_dir = "C:/Users/tokud/OneDrive/Documents/Machine Learning - UCSD Extension Springboard Course/Capstone/Datasets/Food/"
output_dir = "C:/Users/tokud/OneDrive/Documents/Machine Learning - UCSD Extension Springboard Course/Capstone/Datasets/Burritos/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

photo_ids = set(burritos_filtered['photo_id'].tolist())  


for image_file in os.listdir(photos_dir):
    if image_file.endswith('.jpg'):

        photo_id = os.path.splitext(image_file)[0]
        
        if photo_id in photo_ids:
            src_path = os.path.join(photos_dir, image_file)
            dest_path = os.path.join(output_dir, image_file)
            
            shutil.copy(src_path, dest_path)